In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import pandas as pd
import sklearn
import imblearn as imb
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

In [ ]:
print("TensorFlow version:", tf.__version__)
print("NumPy version:", np.__version__)
print("OpenCV version:", cv2.__version__)
print("Python version:", os.sys.version)
print("Pandas version:", pd.__version__)
print("Matplotlib version:", plt.matplotlib.__version__)
print("Scikit-learn version:", sklearn.__version__)
print("Imbalanced-learn version:", imb.__version__)

In [ ]:
DATASET_PATH = '../dataset'
IMAGE_SIZE = (256, 256)

In [ ]:
def load_image(image_path):
  try:
    image = cv2.imread(image_path)
    if image is None:
      raise FileNotFoundError("Failed to load image. Please check the file path.")
    return image
  except Exception as e:
    print("Error:", e)
    return None

In [ ]:
def image_preprocessing(image):
  image = cv2.resize(image, IMAGE_SIZE)
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
  equalized = clahe.apply(gray)
  # equalized = equalized.astype(np.float32) / 255.0
  return image, gray, equalized

In [ ]:
def show_image(image, title):
  plt.imshow(image, cmap='gray')
  plt.title(title)
  plt.axis('off')
  plt.show()

In [ ]:
categories = []
images = []
grays = []
equalizeds = []
labels = []

for dir in os.listdir(path=DATASET_PATH):
    categories.append(dir)
    dir_path = os.path.join(DATASET_PATH, dir)

    for file in os.listdir(dir_path):
        file_path = os.path.join(DATASET_PATH, dir, file)
        image = load_image(file_path)

        if image is not None:
            image, gray, equalized = image_preprocessing(image)
            images.append(image)
            grays.append(gray)
            equalizeds.append(equalized)
            labels.append(categories.index(dir))

In [ ]:
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.imshow(images[0], cmap='gray')
plt.title('Original Image')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(grays[0], cmap='gray')
plt.title('Grayscale Image')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(equalizeds[0], cmap='gray')
plt.title('Equalized Image')
plt.axis('off')
plt.show()

In [ ]:
x = np.array(equalizeds)
y = np.array(labels)
print("Data shape:", x.shape, y.shape)

In [ ]:
x_reshaped = x.reshape(x.shape[0], -1)
print("Reshaped data shape:", x_reshaped.shape)

In [ ]:
smote = SMOTE(random_state=42)
x_resampled, y_resampled = smote.fit_resample(x_reshaped, y)
print("Resampled data shape:", x_resampled.shape, y_resampled.shape)

In [ ]:
x_resampled_reshaped = x_resampled.reshape(-1, IMAGE_SIZE[0], IMAGE_SIZE[1])
x_expanded = np.expand_dims(x_resampled_reshaped, axis=-1)
x_expanded = np.repeat(x_expanded, 3, axis=-1)
print("Resampled reshaped data shape:", x_expanded.shape)

In [ ]:
downsampled_images = [cv2.resize(image, (64, 64)) for image in x_expanded]
x_normalized = np.array(downsampled_images).astype(np.float32) / 255.0


In [ ]:
def show_bar_chart(x_axis, y_axis, title, x_label, y_label):
  plt.figure(figsize=(8, 4))
  bars = plt.bar(x_axis, y_axis)
  plt.title(title)
  plt.xlabel(x_label)
  plt.ylabel(y_label)
  plt.ylim(0, 1200)
  for bar, value in zip(bars, y_axis):
      plt.text(bar.get_x() + bar.get_width() / 2, value + 0.2, str(value), ha='center', va='bottom', fontsize=10)
  plt.show()

In [ ]:
x_axis = [categories.replace('_', '\n') for categories in categories]
y_axis = [len(y[y == i]) for i in range(len(categories))]
show_bar_chart(x_axis, y_axis, 'Original Data Distribution', 'Categories', 'Count')

In [ ]:
x_axis = [categories.replace('_', '\n') for categories in categories]
y_axis = [len(y_resampled[y_resampled == i]) for i in range(len(categories))]
show_bar_chart(x_axis, y_axis, 'SMOTE Data Distribution', 'Categories', 'Count')

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_normalized, y_resampled, test_size=0.2, random_state=42)
print("Train data shape:", x_train.shape, y_train.shape)
print("Test data shape:", x_test.shape, y_test.shape)

In [ ]:
resnet_50 = ResNet50(
    include_top=False
    weights=None,
    input_tensor=None,
    input_shape=(64, 64, 3),
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)